## LLMChain

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
human_prompt = HumanMessagePromptTemplate.from_template(template='Make up a funny name for a company that makes: {product}')

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_prompt])

In [5]:
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                   max_tokens=2000,
                   temperature=0.3)

In [6]:
chain = chat_prompt_template | model

In [7]:
print(chain.invoke(input='Computers').content)

Byte Me Computers!


## Simple Sequential Chain

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Initialize your Chat model
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                   max_completion_tokens=2000,
                   temperature=0.3)

# Define the prompt templates
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
template2 = "Write a blog post using this outline:\n\n{outline}"

# Create ChatPromptTemplates from the templates
prompt1 = ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)

# Define the chains using LCEL (LangChain Expression Language)
chain = (
    {"outline": prompt1 | model | StrOutputParser()}
    | prompt2
    | model
    | StrOutputParser()
)

In [9]:
result = chain.invoke({"topic": "Artificial Intelligence"})

In [10]:
print(result)

# Understanding Artificial Intelligence: A Comprehensive Guide

## Introduction

Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of performing tasks that typically require human intelligence. This includes abilities such as understanding natural language, recognizing patterns, solving problems, and making decisions. In today's world, AI is not just a futuristic concept; it is a driving force behind innovations across various sectors, transforming how we live and work. In this blog post, we will explore the definition of AI, its types, applications, benefits, challenges, and what the future holds for this exciting field.

## 1. What is Artificial Intelligence?

At its core, AI refers to the simulation of human intelligence processes by machines, particularly computer systems. Key components of AI include machine learning, natural language processing, and robotics. 

AI can be categorized into two main types: **Narrow AI** and **General A

## Sequential Chain

### Deprecated syntax

In [11]:
template1 = "Give a summary of this employee's performance review: \n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=model, prompt=prompt1, output_key='review_summary')
template2 = "Identify key employee weaknesses in this review summary: \n{review}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=model, prompt=prompt2, output_key='weaknesses')
template3 = "Create a personalized plan to help and fix these weaknesses: \n{review}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain3 = LLMChain(llm=model, prompt=prompt3, output_key='final_plan')

C:\Users\JimYi\AppData\Local\Temp\ipykernel_17104\1864633277.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=model, prompt=prompt1, output_key='review_summary')


In [12]:
seq_chain = SequentialChain(chains=[chain1, chain2, chain3],
                            input_variables=['review'],
                            output_variables=['review_summary', 'weaknesses', 'final_plan'],
                            verbose=True)

results = seq_chain.invoke("The employee, Joe, is highly motivated. He works hard and write good python codes. However, he struggles with time management and prioritization.")



> Entering new SequentialChain chain...

> Finished chain.


In [13]:
print(results['final_plan'])

To help Joe improve his time management and prioritization skills while leveraging his motivation and coding abilities, we can create a personalized development plan. This plan will include specific goals, strategies, and resources to help him address his weaknesses effectively.

### Personalized Development Plan for Joe

#### Goals:
1. **Improve Time Management Skills**
   - Joe will learn to allocate his time effectively across tasks.
   - He will develop a routine that maximizes productivity.

2. **Enhance Prioritization Skills**
   - Joe will be able to identify high-priority tasks and focus on them first.
   - He will learn to differentiate between urgent and important tasks.

#### Strategies:

1. **Time Management Techniques**
   - **Pomodoro Technique**: Encourage Joe to work in focused sprints (25 minutes of work followed by a 5-minute break) to maintain concentration and prevent burnout.
   - **Time Blocking**: Help Joe create a weekly schedule that allocates specific blocks o

### New syntax

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel

# Initialize your model (replace with your actual model and API key)
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                   max_completion_tokens=2000,
                   temperature=0.3)

# Define prompt templates clearly
template1 = "Give a summary of this employee's performance review:\n{review}"
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
template3 = "Create a personalized plan to help and fix these weaknesses:\n{weaknesses}"

# Create prompt objects
prompt1 = ChatPromptTemplate.from_template(template1)
prompt2 = ChatPromptTemplate.from_template(template2)
prompt3 = ChatPromptTemplate.from_template(template3)

# Define chains using LCEL (LangChain Expression Language)
chain1 = {"review_summary": prompt1 | model | StrOutputParser()}
chain2 = {"weaknesses": prompt2 | model | StrOutputParser()}
chain3 = {"final_plan": prompt3 | model | StrOutputParser()}

In [15]:
# Combine chains sequentially
seq_chain = (
    RunnablePassthrough.assign(**chain1)
    | RunnablePassthrough.assign(**chain2)
    | RunnablePassthrough.assign(**chain3)
)

In [16]:
# Invoke the chain with your input
results = seq_chain.invoke({
    "review": "The employee, Joe, is highly motivated. He works hard and writes good python codes. However, he struggles with time management and prioritization."
})

# Print results
print("Review Summary:\n", results["review_summary"])
print("\nWeaknesses:\n", results["weaknesses"])
print("\nPersonalized Improvement Plan:\n", results["final_plan"])

Review Summary:
 In Joe's performance review, he is recognized for his high motivation and strong work ethic, particularly in writing quality Python code. However, he faces challenges with time management and prioritization, which may impact his overall effectiveness.

Weaknesses:
 The key employee weaknesses identified in the review summary for Joe are:

1. **Time Management**: Joe struggles to manage his time effectively, which can hinder his productivity and ability to meet deadlines.

2. **Prioritization**: He has difficulty prioritizing tasks, which may lead to inefficiencies and affect his overall performance.

These weaknesses suggest that while Joe has strong technical skills and motivation, he needs to improve his organizational skills to enhance his effectiveness in his role.

Personalized Improvement Plan:
 To help Joe address his weaknesses in time management and prioritization, we can create a personalized development plan that includes specific strategies, tools, and acti